In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [2]:
mnist = input_data.read_data_sets('./MNIST_data', one_hot=True)
# define inputs
# discriminator
X = tf.placeholder(tf.float32, shape=[None, 784])
# genarater
Z = tf.placeholder(tf.float32, shape=[None, 100])

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./MNIST_data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [4]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

def generator(z):
    with tf.variable_scope("generator"):
        G_h1 = tf.layers.dense(Z,
                               128,
                               activation=tf.nn.relu,
                               kernel_initializer=tf.glorot_normal_initializer())
        G_prob = tf.layers.dense(G_h1, 784, activation = tf.nn.sigmoid, kernel_initializer=tf.glorot_normal_initializer())
    
        return G_prob

def discriminator(x, reuse):
    with tf.variable_scope("discriminator"):
        D_h1 = tf.layers.dense(x,
                               128,
                               reuse = reuse,
                               activation=tf.nn.relu,
                               kernel_initializer=tf.glorot_normal_initializer(), 
                               name = "L1")
        D_logit = tf.layers.dense(D_h1, 1, reuse = reuse, kernel_initializer=tf.glorot_normal_initializer(), name = "L2")
        D_prob = tf.nn.sigmoid(D_logit)
        return D_prob, D_logit


In [5]:
G_sample = generator(Z)
D_real, D_logit_real = discriminator(X, reuse = None)
D_fake, D_logit_fake = discriminator(G_sample, reuse =True)

In [6]:
t_vars = tf.trainable_variables()
g_vars = [var for var in t_vars if var.name.startswith("generator")]
d_vars = [var for var in t_vars if var.name.startswith("discriminator")]

In [7]:
# D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
# G_loss = -tf.reduce_mean(tf.log(D_fake))

# Alternative losses:
# -------------------
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=d_vars)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=g_vars)

mb_size = 128
Z_dim = 100

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')
    
i = 0

for it in range(50000):
    if it % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
#         fig.show()
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Iter: 0
D loss: 1.318
G_loss: 2.224

Iter: 1000
D loss: 0.0174
G_loss: 6.107

Iter: 2000
D loss: 0.05284
G_loss: 4.573

Iter: 3000
D loss: 0.1321
G_loss: 4.452

Iter: 4000
D loss: 0.1293
G_loss: 5.246

Iter: 5000
D loss: 0.2013
G_loss: 4.03

Iter: 6000
D loss: 0.5438
G_loss: 4.537

Iter: 7000
D loss: 0.3309
G_loss: 3.179

Iter: 8000
D loss: 0.4383
G_loss: 4.104

Iter: 9000
D loss: 0.6609
G_loss: 3.631

Iter: 10000
D loss: 0.3714
G_loss: 3.27

Iter: 11000
D loss: 0.6902
G_loss: 2.903

Iter: 12000
D loss: 0.4561
G_loss: 3.618

Iter: 13000
D loss: 0.672
G_loss: 2.577

Iter: 14000
D loss: 0.575
G_loss: 2.73

Iter: 15000
D loss: 0.6872
G_loss: 2.374

Iter: 16000
D loss: 0.648
G_loss: 2.474

Iter: 17000
D loss: 0.7922
G_loss: 1.91

Iter: 18000
D loss: 0.7825
G_loss: 2.007

Iter: 19000
D loss: 0.6872
G_loss: 2.068

Iter: 20000
D loss: 0.9274
G_loss: 2.059

Iter: 21000
D loss: 0.7151
G_loss: 2.301

Iter: 22000
D loss: 0.6447
G_loss: 2.222

Iter: 23000
D loss: 0.8193
G_loss: 2.034

Iter: 24000
